In [2]:
## PATH settings
import os
import sys
project_root = os.path.abspath(os.path.join(os.getcwd(), '../../'))
sys.path.append(project_root)
COMMON_PATH = os.path.join(project_root, 'common')

In [10]:
import pandas as pd
from common.extractor.alpaca_extractor import AlpacaExtractor
from datetime import datetime
import numpy as np
from scipy.stats import norm

In [4]:
alp = AlpacaExtractor()

In [30]:
ticker_prices = alp.prices("SBEV",datetime(2020,1,1),datetime(2025,5,1))
warrant_prices = alp.prices("SBEV.WS",datetime(2024,5,1),datetime(2025,5,1))

In [13]:
# Black-Scholes formula
def black_scholes(S, K, T, r, sigma, option_type="call"):
    """
    S: Current stock price
    K: Strike price
    T: Time to expiration (in years)
    r: Risk-free interest rate
    sigma: Volatility of the underlying asset
    option_type: "call" for warrants
    
    Returns: Black-Scholes price
    """
    d1 = (np.log(S / K) + (r + (sigma ** 2) / 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option_type == "call":
        price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    else:
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    
    return price

# Inputs
S = 3.58  # Current stock price (SBEV)
K = 184   # Strike price
T = 1     # Time to expiration (assume 1 year)
r = 0.05  # Risk-free rate (5%)
sigma = 0.50  # Estimated volatility (50%)

# Calculate warrant price (adjusting for 1/40th per unit)
warrant_price = black_scholes(S, K, T, r, sigma, "call") / 40

print(f"Estimated warrant price using Black-Scholes: ${warrant_price:.4f}")

Estimated warrant price using Black-Scholes: $0.0000


In [31]:
ticker_prices["returns"] = ticker_prices["adjclose"].pct_change()
sigma = np.std(ticker_prices["returns"].dropna()) * np.sqrt(252)

warrant_price = black_scholes(514,184,1,0.0,sigma,"call")/40
print(warrant_price)

9.554391249983976
